# Đồ án môn Lập trình song song ứng dụng
**Giảng viên: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 1

### Thành viên:
1. 1712284 - Hoàng Gia Bảo
2. 1712685 - Trần Huy Phượng

## 1. Mô tả ứng dụng

Tên ứng dụng: **Phát hiện cộng đồng trong mạng xã hội bằng thuật toán Girvan Newman**

* Input: Đồ thị vô hướng, không trọng số

* Output: Tập hợp các cộng đồng được phát hiện - mỗi cộng đồng được thể hiện là list các node

* Ý nghĩa thực tế: có rất nhiều, ví dụ một số ứng dụng:
  * Tiếp thị: xác định các cụm khách hàng có chung sở thích trong một mạng thể hiện quan hệ giữa người mua và sản phẩm trên sàn thương mại điện tử để xây dựng hệ thống tư vấn hiệu quả
  * Kết bạn, mở rộng mối quan hệ
  * ...


Khái niệm cộng đồng, được định nghĩa là tập các thực thể có những tính chất 
tương tự nhau hoặc cùng đóng một vai trò trong một mạng xã hội. 

Hình dưới đây thể hiện ví dụ về output của bài toán

In [ ]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1COUG0WrM545iqLHDcBYtjD0PkYNc5arQ'/>

Ứng dụng này cần phải được tăng tốc do vì khi recommend 1 sản phẩm cho khách hàng, ta cần thời gian nhanh để đem lại trải nghiệm tốt nhất cho khách hàng. Trong khi đó thuật toán Girvan Newman có thời gian thực thi khá chậm nên việc song song hóa là rất cần thiết.

## 2. Tiền xử lí dữ liệu

### 1. Tải dữ liệu

In [ ]:
!wget https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv

In [ ]:
import csv

In [ ]:
fileData = []
header = []
with open('ub_sample_data.csv', newline ='') as csvfile:
    file = csv.reader(csvfile, delimiter=',')
    for row in file:
        if not header:
            header.append(row)
            continue
        fileData.append(row)
    csvfile.close()

### 2. Tạo graph từ input

#### Tạo graph

In [ ]:
filterThreshold = 7
setUsers = {}

for [user, item] in fileData:
    if user not in setUsers:
        setUsers[user] = {item}
    else:
        setUsers[user].add(item)

gPyObject = {}

# filter and make adjacency list of graph
for user1, item1 in setUsers.items():
    for user2, item2 in setUsers.items():
        if user1 == user2: continue
        if(len(item1 & item2) >= filterThreshold):
            if user1 not in gPyObject:
                gPyObject[user1] = [user2]
            else:
                gPyObject[user1].append(user2)
cnt = 0
for key, val in gPyObject.items():
    print(key, val)
    cnt += 1
    if cnt == 5:
        break

#### Thay đổi định dạng user từ string sang integer

In [ ]:
transferUser = {}
cnt = 0
for key in gPyObject.keys():
    transferUser[key] = cnt
    cnt += 1
g = [[] for _ in range(cnt)]
for key, val in gPyObject.items():
    user1 = transferUser[key]
    for it in val:
        user2 = transferUser[it]
        g[user1].append(user2)
g[:5]

#### Ghi graph ra file

In [ ]:
with open('graph.txt', "w+") as fp:
    fp.write(str(cnt))
    fp.write("\n")
    for i in g:
        line = " ".join([str(item) for item in i])
        fp.write(line)
        fp.write("\n")

    fp.close()

## 3. Cài đặt tuần tự

Các yêu cầu cơ bản của một thuật toán phát hiện cộng đồng nói chung đó là nó phải tự chỉ được ra một phép phân chia tự nhiên giữa các đỉnh mà không đòi hỏi người ta phải xác định số cộng đồng nên có là bao nhiêu, hay giới hạn trong kích thước của các cộng đồng

### 0. Giới thiệu thuật toán Girvan-Newman

- Là thuật toán điển hình nhất trong các thuật toán phân chia đồ thị do nó có  ý nghĩa về mặt lịch sử (đánh dấu bước khởi đầu của một thời kỳ mới trong sự phát triển của phát hiện cộng đồng)
- Đây là một phương pháp chia nhỏ, trong đó các cạnh được loại bỏ dần dần ra khỏi đồ thị
- Các cạnh loại bỏ trong mỗi bước được xác định qua một độ đo trung gian (gọi là betweeness centrality)
- Sau khi loại bỏ mỗi cạnh, độ đo trung gian của các cạnh còn lại sẽ được tính lại dựa vào cạnh bị loại bỏ đó

In [ ]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1lRwfBzDtRg0e7D5UIHQ09r84JeDISF-7'/>

Betweeness Centrality - BC (tạm dịch: mức độ tập trung) là một độ đo để thể hiện mức độ ảnh hưởng của một cạnh đối với luồng thông tin trong đồ thị. Nó thường được sử dụng để tìm các cạnh đóng vai trò là cầu nối từ phần này sang phần khác của đồ thị.

In [ ]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1kv2-5XsuvzbKuCxUkLW31P-AqBWP-DRi'/>

#### Đánh giá cách phân chia cộng đồng

Để đánh giá một cách phân chia cộng đồng là tốt hay không, thì ta sẽ dựa vào độ đo Modularity(Q)

Đồ thị có giá trị Modularity cao sẽ có dày đặc các kết nối giữa các nút trong cùng một cộng đồng, nhưng sẽ có thưa thớt kết nối giữa các nút trong các cộng đồng khác nhau

--> Giá trị Modularity càng cao thì cách phân chia cộng đồng đó càng tốt

Biểu thức toán học độ đo Modularity

In [ ]:
%%html
<img src='https://drive.google.com/uc?export=view&amp;id=1RTdu3AwSZCxJx9lxcQBd-DD337nEOc2E'/>

Trong đó:
- $m$: số lượng cạnh của đồ thị ban đầu
- $A_{ij}$: bằng 1 nếu cạnh i có kết nối với cạnh j, 0 nếu ngược lại (A là ma trận kề biểu diễn đồ thị)
- $k_i$, $k_j$: bậc của đỉnh i và j ở đồ thị ban đầu
- $\delta(C_i, C_j)$: bằng 1 nếu đỉnh i và j cùng thuộc 1 cộng đồng, bằng 0 nếu ngược lại

Miền giá trị của $Q \in [-0.5, 1]$

Trong bài toán thực tế, giá trị Modularity cao nhất thường $\in [0.3, 0.7]$

### 1. Thiết kế

Các bước thực hiện:
1. Xác định độ đo betweenness cho các cạnh trong đồ thị bằng thuật toán Girvan Newman
2. Loại bỏ cạnh có betweenness lớn nhất khỏi graph
3. Thực hiện lại bước 2 đến khi không còn cạnh nào

https://colab.research.google.com/drive/1R2Gt2wPEeEL-9_kGfXJRUy24zv1APglX?usp=sharing

### 2. Đánh giá

#### Kịch bản thí nghiệm

Sử dụng 3 bộ dữ liệu:
- Bộ dữ liệu 1: 
  - 500 đỉnh
  - 75699 cạnh
- Bộ dữ liệu 2:
  - 1000 đỉnh
  - 303263 cạnh
- Bộ dữ liệu 3:
  - 2000 đỉnh
  - 1556977 cạnh

Thực hiện 2 version: \
    + ver1: code python bình thường \
    + ver2: chuyển code về sử dụng mảng numpy và sử dụng decorator @jit(nopython=True)

##### Version 1

In [ ]:
%%time
!python sequence.py

##### Version 2

In [ ]:
%%time
!python sequenceDecoratorJit.py

#### Kiểm tra độ chính xác


Kiểm tra độ chính xác bằng thư viện NetworkX

In [ ]:
%%time
!python testNetworkx.py

#### Nhận xét

- Cài đặt tuần tự có thời gian chạy gần bằng với thư viện networkX: khoảng 20s
- Cài đặt tuần tự có decorator @jit(nopython=True) có thời gian chạy rất nhanh: khoảng 2.5s

## Cài đặt song song

### 1. Thiết kế

### 2. Đánh giá

#### Kịch bản thí nghiệm

Sử dụng chung bộ dữ liệu với cài đặt tuần tự:
- Bộ dữ liệu 1: 
  - 500 đỉnh
  - 75699 cạnh
- Bộ dữ liệu 2:
  - 1000 đỉnh
  - 303263 cạnh
- Bộ dữ liệu 3:
  - 2000 đỉnh
  - 1556977 cạnh

#### Chạy code và xem kết quả

In [ ]:
!nvidia-smi

In [ ]:
%%time
!python parallel.py

#### Kiểm tra độ chính xác

#### Nhận xét

## Tối ưu hóa phiên bản song song

### 1. Thiết kế

Kiểm tra lí do code song song chạy chậm

In [ ]:
!nvprof --print-gpu-trace python parallel.py

--> Code song song chậm do quá trình chuyển dữ liệu từ host sang device, device sang host

### 2. Đánh giá

#### Kịch bản thí nghiệm

Sử dụng chung bộ dữ liệu với cài đặt tuần tự:
- Bộ dữ liệu 1: 
  - 500 đỉnh
  - 75699 cạnh
- Bộ dữ liệu 2:
  - 1000 đỉnh
  - 303263 cạnh
- Bộ dữ liệu 3:
  - 2000 đỉnh
  - 1556977 cạnh

#### Chạy code và xem kết quả

In [ ]:
%%time
!python parallel_optim_v1.py

In [ ]:
!nvprof --print-gpu-trace python parallel_optim_v1.py

In [ ]:
%%time
!python parallel_optim_v2.py

In [ ]:
!nvprof --print-gpu-trace python parallel_optim_v2.py

Tổng hợp thời gian chạy các phiên bản:

Bộ dữ liệu -- Tuần tự -- NetworkX -- Tuần tự jit -- Song song -- Song song tối ưu v1 -- Song song tối ưu v2

    1   --  23s    --   23s    --  3s   --   1m4s   --   13s    --  10s

    2   --  3m24s  --   3m35s  --  9s   --   3m35s  --   25s    --  19s
    
    3   --         --   34m15s --  1m7s --          --   1m5s   --  40s